## basic test

In [1]:
import sys
sys.path.append('../../build')
import libry as ry
import numpy as np
import time

**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback



In [2]:
#-- REAL WORLD configuration, which is attached to the physics engine
# accessing this directly would be cheating!
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/ex01.g")

In [3]:
S = RealWorld.simulation(ry.SimulatorEngine.bullet, True)
S.addSensor("camera")

In [4]:
#-- MODEL WORLD configuration, this is the data structure on which you represent
# what you know about the world and compute things (controls, contacts, etc)
C = ry.Config()
#D = C.view() #rather use the ConfiguratioViewer below
C.addFile("../../scenarios/pandasTable.g")
C.view()

In [5]:
# add a new frame to the MODEL configuration
# (Perception will later have to do exactly this: add perceived objects to the model)
obj = C.addFrame("object")

In [6]:
# set frame parameters, associate a shape to the frame, 
obj.setPosition([.8,0,1.5])
obj.setQuaternion([1,0,.5,0])
obj.setShape(ry.ST.capsule, [.2,.02])
obj.setColor([1,0,1])

from pyquaternion import Quaternion


In [7]:
#-- the following is the simulation loop
tau = .01


#print(C.getFrameNames())
marker = C.addFrame("marker","object")

q0 = Quaternion(axis=[0,0,1],angle=3.14159265 /2)
q1 = Quaternion(axis=[0,1,0],angle=3.14159265 /2)
q= q1*q0
marker.setRelativeQuaternion(q.normalised.elements)


for t in range(300):
    time.sleep(0.01)

    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
            [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow

    #some good old fashioned IK
    C.setJointState(q) #set your robot model to match the real q

    [y,J] = C.evalFeature(ry.FS.poseDiff, ["R_gripper", "marker"])
    vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ (-y);

    #send velocity controls to the simulation
    S.step(vel, tau, ry.ControlMode.velocity)